# Set up EDB for PCB DC IR Analysis
This example shows how to set up the electronics database (EDB) for DC IR analysis from a single
configuration file.

## Import the required packages

In [1]:
import json
import os
import tempfile

from ansys.aedt.core import Hfss3dLayout, Icepak
from ansys.aedt.core.downloads import download_file

from pyedb import Edb

AEDT_VERSION = "2024.2"
NG_MODE = False


Download the example PCB data.

In [2]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source="edb/ANSYS-HSD_V1.aedb", destination=temp_folder.name)

## Load example layout

In [3]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

PyAEDT INFO: Logger is initialized in EDB.


PyAEDT INFO: legacy v0.35.0


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: Database ANSYS-HSD_V1.aedb Opened in 2024.2


PyAEDT INFO: Cell main Opened


PyAEDT INFO: Builder was initialized.


PyAEDT INFO: EDB initialized.


## Create an empty dictionary to host all configurations

In [4]:
cfg = dict()
cfg["sources"] = []

## Update stackup

In [5]:
cfg["stackup"] = {
    "layers": [
        {"name": "Top", "type": "signal", "material": "copper", "fill_material": "FR4_epoxy", "thickness": "0.035mm"},
        {"name": "DE1", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.1mm"},
        {
            "name": "Inner1",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {"name": "DE2", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.088mm"},
        {
            "name": "Inner2",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {"name": "DE3", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.1mm"},
        {
            "name": "Inner3",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {
            "name": "FR4_epoxy-1mm",
            "type": "dielectric",
            "material": "FR4_epoxy",
            "fill_material": "",
            "thickness": "1mm",
        },
        {
            "name": "Inner4",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {"name": "DE5", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.1mm"},
        {
            "name": "Inner5",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {"name": "DE6", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.088mm"},
        {
            "name": "Inner6",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {"name": "DE7", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.1mm"},
        {
            "name": "Bottom",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.035mm",
        },
    ]
}

## Define voltage source

In [6]:
cfg["sources"].append(
    {
        "name": "vrm",
        "reference_designator": "U2",
        "type": "voltage",
        "magnitude": 1,
        "positive_terminal": {"net": "1V0"},
        "negative_terminal": {"net": "GND"},
    }
)

## Define current source

In [7]:
cfg["sources"].append(
    {
        "name": "U1_1V0",
        "reference_designator": "U1",
        "type": "current",
        "magnitude": 10,
        "positive_terminal": {"net": "1V0"},
        "negative_terminal": {"net": "GND"},
    }
)

## Define SIwave DC IR analysis setup

In [8]:
cfg["setups"] = [
    {
        "name": "siwave_1",
        "type": "siwave_dc",
        "dc_slider_position": 1,
        "dc_ir_settings": {"export_dc_thermal_data": True},
    }
]

## Define Cutout

In [9]:
cfg["operations"] = {
    "cutout": {"signal_list": ["1V0"], "reference_list": ["GND"], "extent_type": "ConvexHull", "expansion_size": "20mm"}
}

## Define package for thermal analysis (optional)

In [10]:
cfg["package_definitions"] = [
    {
        "name": "package_1",
        "component_definition": "ALTR-FBGA1517-Ansys",
        "maximum_power": 0.5,
        "therm_cond": 2,
        "theta_jb": 3,
        "theta_jc": 4,
        "height": "1mm",
        "apply_to_all": False,
        "components": ["U1"],
    },
]

## Write configuration into a JSON file

In [11]:
file_json = os.path.join(temp_folder.name, "edb_configuration.json")
with open(file_json, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

## Import configuration into example layout

In [12]:
edbapp.configuration.load(config_file=file_json)

Apply configuration to EDB.

In [13]:
edbapp.configuration.run()

PyAEDT INFO: Cutout Multithread started.


PyAEDT INFO: Net clean up Elapsed time: 0m 2sec


PyAEDT INFO: Extent Creation Elapsed time: 0m 0sec


PyAEDT INFO: 533 Padstack Instances deleted. Elapsed time: 0m 1sec


PyAEDT INFO: 105 Primitives deleted. Elapsed time: 0m 2sec


PyAEDT INFO: 470 components deleted


PyAEDT INFO: Cutout completed. Elapsed time: 0m 5sec


True

Save and close EDB.

In [14]:
edbapp.save()
edbapp.close()

True

The configured EDB file is saved in a temp folder.

In [15]:
print(temp_folder.name)

C:\Users\ansys\AppData\Local\Temp\tmp7lpd2cqa.ansys


## Load edb into HFSS 3D Layout.

In [16]:
h3d = Hfss3dLayout(edbapp.edbpath, version=AEDT_VERSION, non_graphical=NG_MODE, new_desktop=True)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: PyAEDT version 0.14.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_1b6e2079-1936-4396-9384-6e6b4b1ce97b.log is enabled.


...

PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64


PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 version started with process ID 11084.


PyAEDT INFO: EDB folder C:\Users\ansys\AppData\Local\Temp\tmp7lpd2cqa.ansys\edb/ANSYS-HSD_V1.aedb has been imported to project ANSYS-HSD_V1


PyAEDT INFO: Active Design set to 0;main


PyAEDT INFO: Aedt Objects correctly read


## Prepare for electro-thermal analysis in Icepak (Optional)

In [17]:
h3d.modeler.set_temperature_dependence(include_temperature_dependence=True, enable_feedback=True, ambient_temp=22)

PyAEDT INFO: Loading Modeler.


PyAEDT INFO: Modeler loaded.


PyAEDT INFO: EDB loaded.


PyAEDT INFO: Layers loaded.


PyAEDT INFO: Primitives loaded.


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Set the temperature dependence for the design.


PyAEDT INFO: Assigned Objects Temperature


True

## Analyze

In [18]:
h3d.analyze()

PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/HFSS 3D Layout Design correctly changed.


PyAEDT INFO: Solving all design setups.


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/HFSS 3D Layout Design correctly changed.


PyAEDT INFO: Design setup None solved correctly in 0.0h 0.0m 37.0s


True

## Plot DC voltage

In [19]:
voltage = h3d.post.create_fieldplot_layers_nets(
    layers_nets=[
        ["Inner2", "1V0"],
    ],
    quantity="Voltage",
    setup="siwave_1",
)

PyAEDT INFO: Parsing C:/Users/ansys/AppData/Local/Temp/tmp7lpd2cqa.ansys/edb/ANSYS-HSD_V1.aedt.


PyAEDT INFO: File C:/Users/ansys/AppData/Local/Temp/tmp7lpd2cqa.ansys/edb/ANSYS-HSD_V1.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.0312504768371582


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


...

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: Database ANSYS-HSD_V1.aedb Opened in 2024.2


PyAEDT INFO: Cell main Opened


PyAEDT INFO: Builder was initialized.


PyAEDT INFO: EDB initialized.


In [20]:
file_path_image = os.path.join(temp_folder.name, "voltage.jpg")
voltage.export_image(
    full_path=file_path_image,
    width=640,
    height=480,
    orientation="isometric",
    display_wireframe=True,
    selections=None,
    show_region=True,
    show_axis=True,
    show_grid=True,
    show_ruler=True,
)

'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmp7lpd2cqa.ansys\\voltage.jpg'

## Plot power density

In [21]:
power_density = h3d.post.create_fieldplot_layers_nets(
    layers_nets=[
        ["Inner2", "no-net"],
    ],
    quantity="Power Density",
    setup="siwave_1",
)

In [22]:
file_path_image = os.path.join(temp_folder.name, "power_density.jpg")
power_density.export_image(
    full_path=file_path_image,
    width=640,
    height=480,
    orientation="isometric",
    display_wireframe=True,
    selections=None,
    show_region=True,
    show_axis=True,
    show_grid=True,
    show_ruler=True,
)

'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmp7lpd2cqa.ansys\\power_density.jpg'

## Save HFSS 3D Layout project

In [23]:
h3d.save_project()

PyAEDT INFO: Project ANSYS-HSD_V1 Saved correctly


True

## Create an Icepak design

In [24]:
ipk = Icepak(version=AEDT_VERSION, non_graphical=NG_MODE, new_desktop=False)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: PyAEDT version 0.14.dev0.


PyAEDT INFO: Returning found Desktop session with PID 11084!


PyAEDT INFO: No project is defined. Project ANSYS-HSD_V1 exists and has been read.


PyAEDT INFO: No consistent unique design is present. Inserting a new design.


PyAEDT INFO: Added design 'Icepak_O2X' of type Icepak.


PyAEDT INFO: Aedt Objects correctly read


## Create PCB

In [25]:
pcb = ipk.create_ipk_3dcomponent_pcb(
    compName="PCB_pyAEDT",
    setupLinkInfo=[h3d.project_file, h3d.design_name, "siwave_1", True, True],
    solutionFreq=None,
    resolution=0,
    extent_type="Bounding Box",
    powerin="0",
)

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


## Include pckage definition from Edb

In [26]:
pcb.included_parts = "Device"

## Adjust air region

In [27]:
region = ipk.modeler["Region"]
faces = [f.id for f in region.faces]
ipk.assign_pressure_free_opening(assignment=faces, boundary_name="Outlet")

## Setup mesh

In [28]:
glob_msh = ipk.mesh.global_mesh_region
glob_msh.global_region.positive_z_padding_type = "Absolute Offset"
glob_msh.global_region.positive_z_padding = "50 mm"
glob_msh.global_region.negative_z_padding_type = "Absolute Offset"
glob_msh.global_region.negative_z_padding = "80 mm"

PyAEDT INFO: Parsing C:/Users/ansys/AppData/Local/Temp/tmp7lpd2cqa.ansys/edb/ANSYS-HSD_V1.aedt.


PyAEDT INFO: File C:/Users/ansys/AppData/Local/Temp/tmp7lpd2cqa.ansys/edb/ANSYS-HSD_V1.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.03126120567321777


PyAEDT WARNING: No mesh operation found.


PyAEDT WARNING: No mesh region found.


PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing objects from AEDT file


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


PyAEDT INFO: Mesh class has been initialized! Elapsed time: 0m 0sec


In [29]:
glob_msh = ipk.mesh.global_mesh_region
glob_msh.manual_settings = True
glob_msh.settings["EnableMLM"] = True
glob_msh.settings["EnforeMLMType"] = "2D"
glob_msh.settings["2DMLMType"] = "Auto"
glob_msh.settings["MaxElementSizeY"] = "2mm"
glob_msh.settings["MaxElementSizeX"] = "2mm"
glob_msh.settings["MaxElementSizeZ"] = "3mm"
glob_msh.settings["MaxLevels"] = "2"

In [30]:
glob_msh.update()

True

## Place monitor

In [31]:
cpu = ipk.modeler["PCB_pyAEDT_U1_device"]
m1 = ipk.monitor.assign_face_monitor(
    face_id=cpu.top_face_z.id,
    monitor_quantity="Temperature",
    monitor_name="TemperatureMonitor1",
)

## Create Icepak setup

In [32]:
setup1 = ipk.create_setup(MaxIterations=10)

PyAEDT INFO: Key MaxIterations matched internal key 'Convergence Criteria - Max Iterations' with confidence of 52.


Add 2-way coupling to the setup

In [33]:
ipk.assign_2way_coupling(number_of_iterations=1)

True

## Save

In [34]:
ipk.save_project()

PyAEDT INFO: Project ANSYS-HSD_V1 Saved correctly


True

## Shut Down Electronics Desktop

In [35]:
ipk.release_desktop()

PyAEDT INFO: Desktop has been released and closed.


True

All project files are saved in the folder ``temp_file.dir``. If you've run this example as a Jupyter notebook you
can retrieve those project files.